In [ ]:
pip install pulp

In [ ]:
# RUN ONCE TO START
from pulp import *
from itertools import combinations
from collections import defaultdict

# Define all tiles
all_tiles = {}
all_tiles_list = []

for num in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]:
    for col in ['y', 'r', 't', 'b']:
        all_tiles[f'{num}{col}'] = 0
        all_tiles_list.append(f'{num}{col}')

all_tiles['w'] = 0
all_tiles_list.append('w')

# Generate all possible sets
possible_groups = []
for num in range(1, 14):
    for col_comb in [['y', 'r', 't'], ['y', 'r', 'b'], ['y', 't', 'b'],
                     ['t', 'r', 'b'], ['y', 'r', 't', 'b']]:
        group = [f'{num}{col}' for col in col_comb]
        possible_groups.append(group)

possible_runs = []
for num in range(1, 12):
    for col in ['y', 'r', 't', 'b']:
        for length in [3, 4, 5]:
            if (num + length) <= 14:
                run = [f'{num + i}{col}' for i in range(length)]
                possible_runs.append(run)

possible_sets = possible_groups + possible_runs

# Generate sets with wild
wild_sets = []
for s in possible_sets:
    for num_wilds in range(1, 3):
        n = len(s)
        for positions in combinations(range(n), num_wilds):
            new_set = s.copy()
            for pos in positions:
                new_set[pos] = 'w'
            wild_sets.append(new_set)

possible_sets += wild_sets

# Make them tuples
possible_sets = [tuple(s) for s in possible_sets]

# Make tile_counts for ease later
tile_counts = defaultdict(lambda: defaultdict(int))
for s in possible_sets:
    for t in s:
        tile_counts[t][s] += 1

# Make dictionary with all tiles possible for table, rack
rack_tiles = all_tiles.copy()
table_tiles = all_tiles.copy()

# Define functions to add tiles to rack and table
def rack_add(tiles):
    for t in tiles:
        rack_tiles[t] += 1

def table_add(tiles):
    for t in tiles:
        table_tiles[t] += 1

# Value of meld function for first round
def getScore(meld):
    total = 0
    for tile in meld:
        value = int(tile[:-1])
        total += value
    return total

In [ ]:
# GAME FLOW
first_move = True
people = 3


rack_list = []
for _ in range(14):
    new_rack_tile = input("Enter tile drawn: ")
    rack_list.append(new_rack_tile)
rack_add(rack_list)

order = int(input("What number are you in the order of play? "))
turn = 1
curr_objective = 0

while True:
    # initialize model, decision vars
    rummikub_model = pulp.LpProblem("Rummikub", LpMaximize)
    x = {
        s: LpVariable(name=f"table_{i}", lowBound=0, upBound=2, cat="Integer")
        for i, s in enumerate(possible_sets)
    }

    # objective function = most tiles on board
    rummikub_model += (
        pulp.lpSum(
            x[s] * tile_counts[t][s] for s in possible_sets for t in all_tiles_list
        ),
        "MaximizeTilesPlayedFromRack"
    )

    # if it's your turn
    if (turn - order) % people == 0:
        # need meld for first move
        if first_move:
            # only using tiles from rack
            for t in all_tiles:
                rummikub_model += (
                    pulp.lpSum(x[s] * tile_counts[t][s] for s in possible_sets)
                    <= rack_tiles[t]
                )

            rummikub_model.solve()

            # if greater than 30, tiles are played, game starts regular
            total = 0
            played_sets = []
            for s in possible_sets:
                if pulp.value(x[s]) and pulp.value(x[s]) > 0:
                    for _ in range(int(pulp.value(x[s]))):
                        total += getScore(s)
                        played_sets.append(s)

            if total >= 30:
                print(f"Initial meld successful! Played {total} points.")
                print("Play: ")
                for s in played_sets:
                    print(s)
                first_move = False
                for tile_set in played_sets:
                    for tile in tile_set:
                        rack_tiles[tile] -= 1
                        table_tiles[tile] += 1
            else: # draw tile
                print("Go fish")
                new_rack_tile = input("Input new tile: ")
                rack_add([new_rack_tile])

        else:
            # regular turn constraints
            for t in all_tiles:
                # number of times tile is used must be <= number of times in rack or on table
                rummikub_model += (
                    pulp.lpSum(x[s] * tile_counts[t][s] for s in possible_sets)
                    <= rack_tiles[t] + table_tiles[t]
                )

                # number of times tile is used must be >= number times on board
                rummikub_model += (
                    pulp.lpSum(x[s] * tile_counts[t][s] for s in possible_sets)
                    >= table_tiles[t]
                )

            rummikub_model.solve()

            # if there is a move to be made
            if pulp.value(rummikub_model.objective) > curr_objective:

                curr_objective = pulp.value(rummikub_model.objective)

                played_sets = []
                played_tile_counts = all_tiles.copy()

                for s in possible_sets:
                    if pulp.value(x[s]) and pulp.value(x[s]) > 0:
                        count = int(pulp.value(x[s]))
                        for _ in range(count):
                            played_sets.append(s)
                            for tile in s:
                                played_tile_counts[tile] += 1

                for tile in all_tiles_list:
                    new_tile_amt = played_tile_counts[tile] - table_tiles[tile]
                    if new_tile_amt > 0:
                        rack_tiles[tile] -= new_tile_amt
                        table_tiles[tile] += new_tile_amt
                        for _ in range(new_tile_amt):
                            print(f"Playing tile from rack: {tile}")

                print("Sets on board are now:")
                for ps in played_sets:
                    print("  ", ps)

            else:
                print("Go fish")
                new_rack_tile = input("Input new tile: ")
                rack_add([new_rack_tile])

    else:
        opp_tile_list = []
        while True:
            opp_tile = input("Opponent played tile (or enter 'done' to finish): ")
            if opp_tile == 'done':
              break
            opp_tile_list.append(opp_tile)
        table_add(opp_tile_list)

        # make new solution set from opponents tiles, only using board
        for t in all_tiles:
            rummikub_model += (
                pulp.lpSum(x[s] * tile_counts[t][s] for s in possible_sets)
                == table_tiles[t]
            )

        rummikub_model.solve()
        curr_objective = pulp.value(rummikub_model.objective)

    turn += 1
